## 使用example

In [4]:
from my_code.mysionna.channel.torch_version.awgn import AWGN

In [1]:
from sionna.channel.awgn import AWGN 

2024-06-24 01:42:51.760927: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 01:42:51.819464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 01:42:51.819487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 01:42:51.821345: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-24 01:42:51.832426: I tensorflow/core/platform/cpu_feature_guar

In [5]:
import tensorflow as tf
from sionna.channel.apply_ofdm_channel import ApplyOFDMChannel  

# 定义输入
x_real = tf.random.normal([10, 2, 2, 14, 64], dtype=tf.float32)
x_imag = tf.random.normal([10, 2, 2, 14, 64], dtype=tf.float32)
x = tf.complex(x_real, x_imag)

h_freq_real = tf.random.normal([10, 2, 2, 2, 2, 14, 64], dtype=tf.float32)
h_freq_imag = tf.random.normal([10, 2, 2, 2, 2, 14, 64], dtype=tf.float32)
h_freq = tf.complex(h_freq_real, h_freq_imag)

no = tf.constant(0.01, dtype=tf.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)


tf.Tensor(
[[[[[-6.90961897e-01-2.28196120e+00j  5.97984254e-01-2.00734019e+00j
      6.87953949e+00-7.72799104e-02j ...
      1.81363434e-01-2.21655679e+00j -2.77264810e+00-2.82742882e+00j
     -4.61794043e+00-1.94688773e+00j]
    [ 8.56843412e-01+6.89553246e-02j -4.32933211e-01+1.82293689e+00j
      1.22549987e+00-2.81031299e+00j ...
     -3.43482876e+00+2.33896065e+00j  2.30540109e+00+3.56898308e+00j
     -6.64234221e-01-6.70692146e-01j]
    [-6.47106320e-02-4.84198749e-01j -6.24890268e-01-1.79632902e+00j
      3.13963509e+00-9.34856319e+00j ...
     -1.83172631e+00+2.64267468e+00j  3.20523024e+00+3.11773133e+00j
     -1.43995905e+00-2.22722483e+00j]
    ...
    [ 1.25636613e+00+1.20882833e+00j -2.27486753e+00-1.87817538e+00j
      2.18505764e+00-5.72963655e-01j ...
      5.69233060e-01+2.47658992e+00j  1.52017605e+00-1.61251843e+00j
      6.86727464e-01-4.35423285e-01j]
    [-7.02149820e+00+3.68192124e+00j  2.72293830e+00-3.79210472e+00j
      1.78976119e+00+1.16105437e+00j ...
   

2024-06-24 01:44:17.034834: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22251 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:17:00.0, compute capability: 8.9


## explain （tf version）
这段代码定义了一个名为 `ApplyOFDMChannel` 的 TensorFlow Keras 层，用于在频域中应用单抽头 OFDM 信道响应。这是一个自定义的 Keras 层类，可以在 Keras 模型中使用。让我们逐行解释相关代码。

### 导入必要的库
```python
import tensorflow as tf
from sionna.utils import expand_to_rank
from .awgn import AWGN
```
- `tensorflow`: 深度学习库，用于定义和训练神经网络。
- `expand_to_rank`: 假设这是一个实用函数，用于扩展张量的维度。
- `AWGN`: 假设这是一个添加加性高斯白噪声的自定义类。

### 定义 ApplyOFDMChannel 类
```python
class ApplyOFDMChannel(tf.keras.layers.Layer):
    r"""ApplyOFDMChannel(add_awgn=True, dtype=tf.complex64, **kwargs)
    
    ...
    """
```
- `ApplyOFDMChannel` 继承自 Keras 的 `Layer` 类。
- 使用了 Keras 层的构造函数来初始化该类。

#### `__init__` 方法
```python
def __init__(self, add_awgn=True, dtype=tf.complex64, **kwargs):
    super().__init__(trainable=False, dtype=dtype, **kwargs)
    self._add_awgn = add_awgn
```
- `__init__` 方法是类的构造函数。
- `add_awgn`：布尔值，是否添加加性高斯白噪声。
- `dtype`：处理和输出的复杂数据类型，默认为 `tf.complex64`。
- `super().__init__`：调用父类的构造函数，并设置 `trainable=False` 表示该层不可训练。

#### `build` 方法
```python
def build(self, input_shape):
    if self._add_awgn:
        self._awgn = AWGN(dtype=self.dtype)
```
- `build` 方法在第一次使用该层时被调用。
- 如果 `add_awgn` 为 `True`，则初始化 `AWGN` 类的实例。

#### `call` 方法
```python
def call(self, inputs):
    if self._add_awgn:
        x, h_freq, no = inputs
    else:
        x, h_freq = inputs

    # Apply the channel response
    x = expand_to_rank(x, h_freq.shape.rank, axis=1)
    y = tf.reduce_sum(tf.reduce_sum(h_freq * x, axis=4), axis=3)

    # Add AWGN if requested
    if self._add_awgn:
        y = self._awgn((y, no))

    return y
```
- `call` 方法是层的核心逻辑。
- `inputs` 是一个元组，可以是 `(x, h_freq, no)` 或 `(x, h_freq)`：
  - `x`：OFDM 输入信号。
  - `h_freq`：频域信道响应。
  - `no`：噪声功率。
- 根据 `add_awgn` 的值，解包输入元组。
- `expand_to_rank`：扩展 `x` 的维度以匹配 `h_freq` 的秩（rank）。
- `h_freq * x`：对输入信号应用频域信道响应。
- `tf.reduce_sum(tf.reduce_sum(h_freq * x, axis=4), axis=3)`：沿特定轴求和以获得输出信号 `y`。
- 如果 `add_awgn` 为 `True`，则调用 `AWGN` 实例添加噪声。
- 返回经过信道和（可选）噪声处理后的输出信号 `y`。

### 总结
- `ApplyOFDMChannel` 类用于在频域中应用单抽头信道响应，并可选地添加噪声。
- 该类继承自 `tf.keras.layers.Layer`，包含 `__init__`、`build` 和 `call` 方法。
- `__init__` 方法初始化层的配置。
- `build` 方法在第一次使用该层时被调用，初始化 AWGN 类的实例。
- `call` 方法实现层的核心逻辑，对输入信号应用信道响应并可选地添加噪声。

## explain （pytorch version）

将 `ApplyOFDMChannel` 层从 TensorFlow 改写成 PyTorch 版本如下：



### 定义 ApplyOFDMChannel 类
```python
class ApplyOFDMChannel(nn.Module):
    def __init__(self, add_awgn=True, dtype=torch.complex64):
        super(ApplyOFDMChannel, self).__init__()
        self.add_awgn = add_awgn
        self.dtype = dtype

        if self.add_awgn:
            self.awgn = AWGN(dtype=self.dtype)

    def forward(self, inputs):
        if self.add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.dim(), axis=1)
        y = torch.sum(torch.sum(h_freq * x, dim=4), dim=3)

        # Add AWGN if requested
        if self.add_awgn:
            y = self.awgn((y, no))

        return y
```

### 测试代码
```python
# 定义输入
x_real = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x_imag = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x = torch.complex(x_real, x_imag)

h_freq_real = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq_imag = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq = torch.complex(h_freq_real, h_freq_imag)

no = torch.tensor(0.01, dtype=torch.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)
```

### 代码解释
1. **导入库**：
   - `torch` 和 `torch.nn` 是 PyTorch 的核心库。
   - `torch.nn.functional` 是用于定义操作和激活函数的库。

2. **定义 `expand_to_rank` 函数**：
   - 这个函数将张量 `x` 扩展到指定的目标秩 `target_rank`。

3. **定义 `AWGN` 类**：
   - `AWGN` 类添加加性高斯白噪声。
   - `forward` 方法将噪声添加到输入信号 `y` 中。

4. **定义 `ApplyOFDMChannel` 类**：
   - `__init__` 方法初始化层的配置，包括是否添加噪声和数据类型。
   - `forward` 方法实现层的核心逻辑，对输入信号应用信道响应并可选地添加噪声。

5. **测试代码**：
   - 定义输入信号 `x` 和频域信道响应 `h_freq`。
   - 创建 `ApplyOFDMChannel` 类的实例并调用 `forward` 方法。

这样就完成了将 `ApplyOFDMChannel` 从 TensorFlow 改写成 PyTorch 的代码。

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from my_code.mysionna.channel.torch_version.awgn import AWGN
from my_code.mysionna.channel.torch_version.utils import expand_to_rank




In [7]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from my_code.mysionna.channel.torch_version.awgn import AWGN
from my_code.mysionna.channel.torch_version.utils import expand_to_rank
class ApplyOFDMChannel(nn.Module):
    def __init__(self, add_awgn=True, dtype=torch.complex64):
        super(ApplyOFDMChannel, self).__init__()
        self.add_awgn = add_awgn
        self.dtype = dtype
        if self.add_awgn:
            self.awgn = AWGN(dtype=self.dtype)

    def forward(self, inputs):
        if self.add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.dim(), axis=1)
        y = torch.sum(torch.sum(h_freq * x, dim=4), dim=3)

        # Add AWGN if requested
        if self.add_awgn:
            y = self.awgn((y, no))

        return y

In [8]:
x_real = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x_imag = torch.randn([10, 2, 2, 14, 64], dtype=torch.float32)
x = torch.complex(x_real, x_imag)

h_freq_real = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq_imag = torch.randn([10, 2, 2, 2, 2, 14, 64], dtype=torch.float32)
h_freq = torch.complex(h_freq_real, h_freq_imag)

no = torch.tensor(0.01, dtype=torch.float32)  # 示例噪声功率

# 创建 ApplyOFDMChannel 实例
apply_ofdm_channel = ApplyOFDMChannel()

# 调用实例
y = apply_ofdm_channel((x, h_freq, no))

print(y)

tensor([[[[[-3.8177e+00-1.2821e+00j, -1.1599e+00-2.3063e+00j,
             3.1329e+00+1.9320e+00j,  ...,
             1.0666e+00-1.2203e+00j, -4.0465e+00+9.0676e-01j,
            -1.8019e-01-7.1328e-01j],
           [-3.7369e+00-6.2695e-01j,  1.6923e+00-3.2801e+00j,
             1.1197e+00-1.7751e+00j,  ...,
            -5.5646e-01-1.8146e-01j, -2.2197e-02-2.0415e+00j,
             1.2568e+00-5.6504e+00j],
           [ 3.0163e-01+3.1853e+00j, -5.2969e+00-3.3031e+00j,
             1.5931e+00+1.4718e+00j,  ...,
            -4.0877e-02-9.5089e-02j,  2.9950e+00-1.9551e+00j,
             6.8650e-01+3.7400e+00j],
           ...,
           [-3.0492e+00+2.6134e+00j, -3.3921e+00+3.5240e-01j,
            -1.8059e+00+6.9555e-01j,  ...,
            -5.2527e+00+2.3452e+00j, -1.7070e+00-1.5006e+00j,
            -3.8428e-01+1.2244e+00j],
           [-3.0882e+00+4.5370e+00j,  1.8301e+00+1.6190e+00j,
             2.7987e+00-3.4584e+00j,  ...,
             2.4224e+00+1.7175e+00j,  1.3329e+00-4.9137e-01